<a href="https://colab.research.google.com/github/luizaclara/KNNtitanic/blob/main/Trabalho_02ICD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://www.kaggle.com/c/titanic/data

https://github.com/datasciencedojo/datasets/blob/master/titanic.csv

PassengerId
Survived
Pclass
Name
Sex
Age
SibSp
Parch
Ticket
Fare
Cabin
Embarked

In [57]:
import sklearn
import pandas as pd
from sklearn.metrics.pairwise import euclidean_distances
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder

In [58]:
class KNN ():
  def __init__(self,k=3):
    self.k = k

  def fit(self,X,Y):
    self.X = X
    self.Y = Y

  def predict_one(self,x):
    distances = sorted(self.distance(x),lambda d:d [0])[:self.k]

    tags_count = {} # {tag : (contagem , soma das distancias)}
    
    for dist, tag in distances:
      if tag not in tags_count:
        count[tag] = (0, 0)
      count[tag] = (count[tag][0] + 1, count[tag][1] + dist )

    # ordena em ordem decrescente a contagem e em ordem crescente a distância
    sorted_count = sorted(tags_count.items(), reverse = True, key= lambda x: (x[1][0], -x[1][1]))
    
    return sorted_count[0][0] # nome da categoria com mais contagem e menos distância

  def distance(self, point):
    return list(zip(euclidean_distances(self.X, [point]), self.Y))


In [86]:
dataframe = pd.read_csv("titanic.csv")

In [88]:
# substituir valores nulos

# preenche a idade com a média das idades
dataframe['Age'] = dataframe['Age'].fillna(dataframe['Age'].mean()) 

#preenche o sexo com o que aparece mais vezes
# dataframe['Sex'] = dataframe['Sex'].fillna(dataframe['Sex'].mode())

# drop no 'Embarked' porque tem poucos valores NaN
dataframe.dropna(subset='Embarked', inplace=True)

In [90]:
dataframe.reset_index()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.000000,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.000000,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.000000,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.000000,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.000000,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.000000,0,0,211536,13.0000,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.000000,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,29.699118,1,2,W./C. 6607,23.4500,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.000000,0,0,111369,30.0000,C148,C


In [77]:
# Transformar a coluna 'Sex' e 'Embarked' em duas colunas
encoder = OneHotEncoder(sparse_output=False)
encoded_data = encoder.fit_transform(dataframe[['Sex', 'Embarked']])
encoded_dataframe = pd.DataFrame(encoded_data, columns=encoder.get_feature_names_out(['Sex', 'Embarked']))

In [54]:
# juntar os dataframes
dataframe = pd.concat([dataframe, encoded_dataframe])

In [53]:
columns = ['Pclass', 'Age', 'SibSp', 'Parch', 'Fare'] + list(encoded_dataframe.columns)
y = ['Survived']

In [55]:
X_train, y_train, X_test, y_test = train_test_split( dataframe[[col for col in columns]], dataframe[y], test_size=0.25, random_state=42)

In [56]:
X_train

,Pclass,Age,SibSp,Parch,Fare,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S,Embarked_nan
266,3.0,16.000000,4.0,1.0,39.6875,NaN,NaN,NaN,NaN,NaN,NaN
148,2.0,36.500000,0.0,2.0,26.0000,NaN,NaN,NaN,NaN,NaN,NaN
575,3.0,19.000000,0.0,0.0,14.5000,NaN,NaN,NaN,NaN,NaN,NaN
879,NaN,NaN,NaN,NaN,NaN,1.0,0.0,1.0,0.0,0.0,0.0
888,3.0,29.699118,1.0,2.0,23.4500,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
239,NaN,NaN,NaN,NaN,NaN,0.0,1.0,0.0,0.0,1.0,0.0
403,NaN,NaN,NaN,NaN,NaN,0.0,1.0,0.0,0.0,1.0,0.0
860,3.0,41.000000,2.0,0.0,14.1083,NaN,NaN,NaN,NaN,NaN,NaN
568,NaN,NaN,NaN,NaN,NaN,0.0,1.0,1.0,0.0,0.0,0.0
